Decided to use PyTorch. Keras was giving me quite a headache with it's abstractions.

In [2]:
import torch

from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("./lesion-csv.csv")
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

# All we care about at this point is the id and class
train_df = train_df.drop(["Unnamed: 0", "Unnamed: 0.1", "teethNumbers", "description", "numberOfCanals", "date", "sequenceNumber"], axis=1)
test_df = test_df.drop(["Unnamed: 0", "Unnamed: 0.1", "teethNumbers", "description", "numberOfCanals", "date", "sequenceNumber"], axis=1)

partition = {
    "train": train_df.imageId.values,
    "validation": test_df.imageId.values,
}

In [4]:
n = 65
print(train_df.iloc[n, 0])
print(train_df.iloc[n].imageId)

0130-1
0130-1


In [5]:
class TeethLesionDataset(Dataset):
    """Teeth Lesion dataset."""
    
    def __init__(self, train_frame, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with image ids and labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.lesions_train_frame = train_frame
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.lesions_train_frame)
    
    def __getitem__(self, idx):
        image_id = self.lesions_train_frame.iloc[idx].imageId
        image = cv2.imread(self.root_dir + image_id + '.jpg')
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
        label = self.lesions_train_frame.iloc[idx].lesion
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [6]:
lesion_dataset = TeethLesionDataset(train_df, './lesion_images/all_images_processed/', transforms.ToTensor())
validation_set = TeethLesionDataset(test_df, './lesion_images/all_images_processed/', transforms.ToTensor())

In [7]:
dataloader = DataLoader(lesion_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i_batch, sample_batched in enumerate(dataloader):
    images, labels = sample_batched
    print(i_batch, images[0].shape)

    # observe 4th batch and stop.
    if i_batch == 3:
        break

0 torch.Size([3, 224, 224])
1 torch.Size([3, 224, 224])
2 torch.Size([3, 224, 224])
3 torch.Size([3, 224, 224])


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.062
[1,   200] loss: 0.037
[2,   100] loss: 0.038
[2,   200] loss: 0.036
Finished Training


In [11]:
testloader = DataLoader(validation_set, batch_size=4,
                        shuffle=True, num_workers=4)
    
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 49 %


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0
